In [0]:
dbutils.fs.mount(
   source = "wasbs://gold@mrkdatalake123.blob.core.windows.net",
   mount_point = "/mnt/gold",
   extra_configs = {"fs.azure.account.key.mrkdatalake123.blob.core.windows.net":"GGaxfo5HiZTDHOZAW6P+Gw1Y5AG70fznqoFeekBc6b0YWIUMAe3Wgy/y4Vy8ch5sWyONnuz3kVX4+AStec4m9A=="})

Out[2]: True

In [0]:
dbutils.fs.ls("/mnt/gold")

Out[4]: []

In [0]:
table_name=[]

for i in dbutils.fs.ls('mnt/silver/SalesLT/'):
    table_name.append(i.name.split('/')[0])

print(table_name)

['Address', 'Customer', 'CustomerAddress', 'Product', 'ProductCategory', 'ProductDescription', 'ProductModel', 'ProductModelProductDescription', 'SalesOrderDetail', 'SalesOrderHeader']


In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col,regexp_replace


for i in  table_name:
    path='/mnt/silver/SalesLT/'+i
    df=spark.read.format('delta').load(path)
    column_names = df.columns
    
    for old_col in column_names:
        new_col="".join(["_" + char if char.isupper() and not old_col[i-1].isupper() else char for i,char in enumerate (old_col)]).lstrip("_")
        df=df.withColumnRenamed(old_col,new_col)

    output_path= '/mnt/gold/SalesLT/'+i+'/'
    df.write.format('delta').mode("overwrite").save(output_path)

